In [16]:
import logging
import os
import pickle
import time
from functools import partial

import numpy as np
import torch
import arch, brax
from datasets import get_dataset
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from pathlib import Path

import jax
import jax.numpy as jnp
from jax import jit, vmap
from jax.example_libraries import optimizers, stax
from jax.tree_util import tree_map
from utils import get_calibration, jaxRNG

In [13]:
class CIFAR10C(Dataset):
    def __init__(self, data_dir, transform=None, severity_level=1):
        # Validate severity level
        if severity_level not in range(1, 6):
            raise ValueError("Severity level must be between 1 and 5.")
            
        self.data_files = sorted([f for f in Path(data_dir).glob('*.npy') if 'labels.npy' not in str(f)])
        self.labels = np.load(Path(data_dir) / 'labels.npy')
        self.transform = transform
        self.severity_level = severity_level
        # Each severity level has 10000 images
        self.images_per_level = 10000
        self.start_idx = self.images_per_level * (severity_level - 1)

    def __len__(self):
        # Assuming same number of images per severity level
        return self.images_per_level

    def __getitem__(self, idx):
        # Adjust idx based on severity level
        adjusted_idx = self.start_idx + idx
        # Assuming data_files are in the correct order
        data = np.load(self.data_files[adjusted_idx // self.images_per_level])[adjusted_idx % self.images_per_level]
        label = self.labels[adjusted_idx % self.images_per_level]
        
        # Apply transformations
        if self.transform:
            data = self.transform(data)

        return data, label

In [17]:
# Define all the args
model= 'sdebnn' #'psdebnn'
print("Using model:", model)
output='output-sdebnn'
seed=0
stl=False
lr=0.0007
epochs=100
bs=128
test_bs=1000
nsamples=1
w_init=-1.0
b_init=-1.0
p_init=-1.0
pause_every=200
no_drift=False
ou_dw=True
kl_coef=0.001
diff_coef=0.1
ds='cifar10'
severity_level = 3 # CHANGE ACCORDINGLY
no_xt=True
acc_grad=1
aug=0
remat=False
ema=0.999
meanfield_sdebnn=False
infer_w0=False
w0_prior_std=0.1
disable_test=False
verbose=True
nblocks='2-2-2'
block_type=0
fx_dim=64
fx_actfn='softplus'
fw_dims='2-128-2'
fw_actfn='softplus'
lr_sched='constant'
rng_generator = jaxRNG(seed= seed)

ode_first=True
timecut=0.1
method_ode='euler'

if ds == 'cifar10':
    print("Using CIFAR-10 dataset")
    train_loader, train_eval_loader, val_loader, test_loader, input_size, train_size = get_dataset(bs, test_bs, "cifar10")
    num_batches = len(test_loader)
    print(f"Number of test batches: {num_batches}")

elif ds == 'cifar10c':
    print("Using CIFAR-10-C dataset at severity level", severity_level)
    # Need to download manually from https://zenodo.org/records/2535967/files/CIFAR-10-C.tar?download=1
    data_dir = "data/cifar10c"

    # Define your transform
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])

    cifar10c_dataset = CIFAR10C(data_dir=data_dir, transform=transform, severity_level=severity_level)

    # Create test DataLoader for the CIFAR-10-C dataset
    #test_loader = DataLoader(cifar10c_dataset, batch_size=test_bs, shuffle=True)
    #num_batches = len(test_loader)
    #print(f"Number of test batches: {num_batches}")

    # Since DataLoader will iterate over the dataset, we can just grab the first 1000 images
    # during the actual testing loop. If we want to explicitly create a smaller dataset:

    indices = torch.randperm(len(cifar10c_dataset)).tolist()
    test_subset_indices = indices[:1000]  # Get the first 1000 indices after shuffling
    test_subset = torch.utils.data.Subset(cifar10c_dataset, test_subset_indices)
    test_loader = DataLoader(test_subset, batch_size=test_bs, shuffle=False)
    num_batches = len(test_loader)
    print(f"Number of test batches: {num_batches}")
else:
    raise ValueError("Dataset not supported!")
    
# SDEBNN specific
mf = partial(brax.MeanField, disable=True) if kl_coef == 0. else brax.MeanField
fw_dims = list(map(int, fw_dims.split("-")))
layers = [mf(arch.Augment( aug))]
nblocks = list(map(int, nblocks.split("-")))
opt_init, opt_update, get_params = optimizers.adam(7e-4)

# Load the checkpoint if it exists
checkpoint_path = os.path.join(output, 'best_model_checkpoint.pkl')
if os.path.exists(checkpoint_path):
    logging.warning("Loading checkpoints...")
    with open(checkpoint_path, "rb") as f:
        checkpoint = pickle.load(f)

    # Extract states from the checkpoint
    start_epoch = checkpoint['epoch']
    best_val_acc = checkpoint['best_val_acc']
    global_step = checkpoint['global_step']
    opt_state = checkpoint['optimizer_state']
    ema_params = checkpoint['ema_state']
    params = checkpoint['model_state']  # Loaded parameters
    print(f"Successfully loaded checkpoints for epoch {start_epoch} with best validation accuracy {best_val_acc}")
else:
    raise SystemExit("No checkpoint found!")

Using model: sdebnn
Using CIFAR-10-C dataset at severity level 3
Number of test batches: 1
Successfully loaded checkpoints for epoch 95 with best validation accuracy 0.7594000101089478


In [18]:
def _nll(params, batch, rng):
    inputs, targets = batch
    preds, kl, info_dict = _predict(params, inputs, rng=rng, full_output=False)
    nll = -jnp.mean(jnp.sum(preds * targets, axis=1))
    return preds, nll, kl, info_dict


@partial(jit, static_argnums=(3,))
def sep_loss(params, batch, rng, kl_coef):  # no backprop
    preds, nll, kl, _ = _nll(params, batch, rng)
    if kl_coef > 0:
        obj_loss = nll + kl * kl_coef
    else:
        obj_loss = nll
    _sep_loss = {'loss': obj_loss, 'kl': kl, 'nll': nll, 'preds': preds}
    return obj_loss, _sep_loss

@partial(jit, static_argnums=(3,))
def loss(params, batch, rng, kl_coef):  # backprop so checkpoint
    _, nll, kl, _ = jax.checkpoint(_nll)(params, batch, rng)
    if kl_coef > 0:
        return nll + kl * kl_coef
    else:
        return nll

@jit
def predict(params, inputs, rng): 
    return _predict(params, inputs, rng=rng, full_output=True)

@partial(jit, static_argnums=(2,))
def accuracy(params, data, nsamples, rng):
    inputs, targets = data
    target_class = jnp.argmax(targets, axis=1)
    rngs = jax.random.split(rng, nsamples)
    preds, _, info_dic = vmap(predict, in_axes=(None, None, 0))(params, inputs, rngs)
    preds = jnp.stack(preds, axis=0)
    avg_preds = preds.mean(0)
    predicted_class = jnp.argmax(avg_preds, axis=1)
    n_correct = jnp.sum(predicted_class == target_class)
    n_total = inputs.shape[0]
    wts = info_dic[model+'_w'] 
    wts = jnp.stack(wts, axis=0)
    avg_wts = wts.mean(0)
    return n_correct, n_total, avg_preds, avg_wts

def update_ema(ema_params, params, momentum=0.999):
    return tree_map(lambda e, p: e * momentum + p * (1 - momentum), ema_params, params)

def evaluate(params, data_loader, input_size, nsamples, rng_generator, kl_coef):
    n_total = 0
    n_correct = 0
    nll = 0
    kl = 0
    logits = np.array([])
    wts = np.array([])
    labels = np.array([])
    for inputs, targets in data_loader:
        targets = jax.nn.one_hot(jnp.array(targets), num_classes=10)
        inputs = jnp.array(inputs).reshape((-1,) + (input_size[-1],) + input_size[:2])
        inputs = jnp.transpose(inputs, (0, 2, 3, 1))  # Permute from NCHW to NHWC
        batch_correct, batch_total, _logits, _wts = accuracy(
            params, (inputs, targets), nsamples, rng_generator.next()
        )
        n_correct = n_correct + batch_correct
        _, batch_nll, batch_kl, _ = jit(_nll)(params, (inputs, targets), rng_generator.next())
        if n_total == 0:
            logits = np.array(_logits)
            wts = np.array(_wts)
            labels = np.array(targets)
        else:
            logits = np.concatenate([logits, np.array(_logits)], axis=0)
            wts = np.concatenate([wts, np.array(_wts)], axis=0)
            labels = np.concatenate([labels, targets], axis=0)
        n_total = n_total + batch_total
        nll = nll + batch_nll
        kl = kl + batch_kl
    return n_correct / n_total, jnp.stack(logits, axis=0), labels, nll / n_total, kl / n_total, jnp.stack(wts, axis=0)

nsteps = 30 # CHANGE FOR EACH EXPERIMENT
print("Starting experiment")

if model == "sdebnn":
    for i, nb in enumerate(nblocks):
        fw = arch.MLP(fw_dims, actfn=fw_actfn, xt=no_xt, ou_dw=ou_dw, nonzero_w=w_init, nonzero_b=b_init, p_scale=p_init)  # weight network is time dependent
        if meanfield_sdebnn:
            layers.extend([mf(brax.SDEBNN(block_type,
                                            fx_dim,
                                            fx_actfn,
                                            fw,
                                            diff_coef=diff_coef,
                                            stl=stl,
                                            xt=no_xt,
                                            nsteps=nsteps,
                                            remat=remat,
                                            w_drift=not no_drift,
                                            stax_api=True,
                                            infer_initial_state=infer_w0,
                                            initial_state_prior_std=w0_prior_std)) for _ in range(nb)
            ])
        else:
            layers.extend([brax.SDEBNN( block_type,
                                        fx_dim,
                                        fx_actfn,
                                        fw,
                                        diff_coef=diff_coef,
                                        stl=stl,
                                        xt=no_xt,
                                        nsteps=nsteps,
                                        remat=remat,
                                        w_drift=not no_drift,
                                        infer_initial_state=infer_w0,
                                        initial_state_prior_std=w0_prior_std) for _ in range(nb)
            ])
        if i < len(nblocks) - 1:
            layers.append(mf(arch.SqueezeDownsample(2)))
    layers.append(mf(stax.serial(stax.Flatten, stax.Dense(10), stax.LogSoftmax)))

    init_random_params, _predict = brax.bnn_serial(*layers)

elif model == "psdebnn":
    for i, nb in enumerate(nblocks):
        fw = arch.MLP(fw_dims, actfn=fw_actfn, xt=no_xt, ou_dw=ou_dw, nonzero_w=w_init, nonzero_b=b_init, p_scale=p_init)  # weight network is time dependent
        if meanfield_sdebnn:
            layers.extend([mf(brax.PSDEBNN( block_type,
                                            fx_dim,
                                            fx_actfn,
                                            fw,
                                            diff_coef=diff_coef,
                                            stl=stl,
                                            xt=no_xt,
                                            nsteps=nsteps,
                                            remat=remat,
                                            w_drift=not no_drift,
                                            stax_api=True,
                                            infer_initial_state=infer_w0,
                                            initial_state_prior_std=w0_prior_std,
                                            ode_first=ode_first,
                                            timecut=timecut,
                                            method_ode=method_ode)) for _ in range(nb)
            ])
        else:
            layers.extend([brax.PSDEBNN(block_type,
                                        fx_dim,
                                        fx_actfn,
                                        fw,
                                        diff_coef=diff_coef,
                                        stl=stl,
                                        xt=no_xt,
                                        nsteps=nsteps,
                                        remat=remat,
                                        w_drift=not no_drift,
                                        infer_initial_state=infer_w0,
                                        initial_state_prior_std=w0_prior_std,
                                        ode_first=ode_first,
                                        timecut=timecut,
                                        method_ode=method_ode) for _ in range(nb)
            ])
        if i < len(nblocks) - 1:
            layers.append(mf(arch.SqueezeDownsample(2)))
    layers.append(mf(stax.serial(stax.Flatten, stax.Dense(10), stax.LogSoftmax)))

    init_random_params, _predict = brax.bnn_serial(*layers)

# for inputs, targets in tqdm(test_loader): # evaluate already deals with the loop
start_time = time.time()
acc, logits, targets, nll, _, _ = evaluate(params, test_loader, input_size, nsamples, rng_generator, kl_coef=kl_coef)

# Calculate inference time
inference_time = time.time() - start_time

# Convert logits to probabilities
probabilities = jax.nn.softmax(logits)
    
# Calculate ECE
# TODO: might be better to use utils.ECE or utils.compute_acc_bin
cal = get_calibration(targets, probabilities)

print(f"Inference time with {nsteps} steps: {inference_time} (s)")
print(f"Accuracy with {nsteps} steps: {acc}")
print(f"NLL with {nsteps} steps: {nll}")
print(f"ECE with {nsteps} steps: {cal['ece']}")

print("Experiment run sucessfully!")

Starting experiment
there were [  0.   0.   2.  30.  66.  82.  84.  85. 130. 520.] items in bins 1-10
Inference time with 30 steps: 38.07104778289795 (s)
Accuracy with 30 steps: 0.7870000600814819
NLL with 30 steps: 0.0006241790251806378
ECE with 30 steps: 0.0401261042443063
Experiment run sucessfully!
